In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

In [ ]:
!unzip '/content/drive/MyDrive/dataset52a7b21.zip'

unzip:  cannot find or open /content/drive/MyDrive/dataset52a7b21.zip, /content/drive/MyDrive/dataset52a7b21.zip.zip or /content/drive/MyDrive/dataset52a7b21.zip.ZIP.


In [2]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import xgboost, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

In [3]:
data = pd.read_csv('/content/drive/MyDrive/Amazon ML Challenge/dataset/train.csv', escapechar = "\\", quoting = csv.QUOTE_NONE)

In [ ]:
data.head()

,TITLE,DESCRIPTION,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","Pete the Cat is the coolest, most popular cat ...","[Pete the Cat Bedtime Blues plush doll,Based o...",MerryMakers,0
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",The New Yorker Handsome Cello Wrapped Hard Mag...,[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1
2,The Ultimate Self-Sufficiency Handbook: A Comp...,NaN,Skyhorse Publishing,imusti,2
3,Amway Nutrilite Kids Chewable Iron Tablets (100),NaN,"[Nutrilite Kids,Chewable Iron Tablets,Quantity...",Amway,3
4,Teacher Planner Company A4 6 Lesson Academic T...,NaN,NaN,NaN,4


In [ ]:
data.shape

(2903024, 5)

In [4]:
a = data['BROWSE_NODE_ID'].value_counts(ascending = True)
map = dict(a)
#ones = data[data['BROWSE_NODE_ID'].map(lambda x: map[x]) == 1]
multiples =  data[data['BROWSE_NODE_ID'].map(lambda x: map[x]) >= 500]

In [5]:
multiples.shape

(2357836, 5)

In [ ]:
len(multiples['BROWSE_NODE_ID'].unique())

863

In [6]:
c, d = model_selection.train_test_split(multiples, train_size = 500000, random_state = 42, stratify = multiples['BROWSE_NODE_ID'])

In [7]:
len(c['BROWSE_NODE_ID'].unique())

863

In [8]:
c.shape

(500000, 5)

In [9]:
targets = c['BROWSE_NODE_ID']

In [10]:
categories = list(targets.unique())
map_targets_to_cat = dict()
map_cat_to_targets = dict()
for i in range(len(categories)):
  map_targets_to_cat[categories[i]] = i
  map_cat_to_targets[i] = categories[i]

In [11]:
cat = targets.map(lambda x: map_targets_to_cat[x])

In [12]:
features = c['TITLE'].replace(np.nan, '', regex = True) + ' ' + c['DESCRIPTION'].replace(np.nan, '', regex = True) + ' ' + c['BULLET_POINTS'].replace(np.nan, '', regex = True)

In [13]:
new_data = pd.DataFrame({'features' : features, 'cat' : cat})

In [14]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(new_data, train_size = 400000, stratify = new_data['cat'], random_state = 42)

In [15]:
X_train = train['features']
X_test = test['features']

y_train = train['cat']
y_test = test['cat']

In [16]:
tokenizer = keras.preprocessing.text.Tokenizer(num_words = 2500)
tokenizer.fit_on_texts(X_train)

In [17]:
len(tokenizer.word_index)

406376

In [ ]:
encoded_docs = tokenizer.texts_to_sequences(X_train)

In [ ]:
padded_docs = sequence.pad_sequences(encoded_docs, padding = 'post')

In [ ]:
X_train_encoded = np.array(padded_docs)

In [ ]:
encoded_docs = tokenizer.texts_to_sequences(X_test)

In [ ]:
padded_docs = sequence.pad_sequences(encoded_docs, padding = 'post')

In [ ]:
X_test_encoded = np.array(padded_docs)

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Embedding(2500, 1024, input_shape = [None], mask_zero = True))
model.add(keras.layers.GRU(1024, return_sequences = True))
model.add(keras.layers.GRU(1024))
model.add(keras.layers.Dense(863, activation = 'softmax'))

In [ ]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = keras.optimizers.Adam(learning_rate = 0.001), metrics = ['accuracy'])

In [ ]:
model.fit(X_train_encoded, y_train, validation_split = 0.1, batch_size = 16, initial_epoch = 0, epochs = 2)

In [ ]:
model.evaluate(X_test_encoded, y_test)

1148/3125 [==========>...................] - ETA: 4:32 - loss: 1.3576 - accuracy: 0.6943

## Submissions

In [20]:
model = keras.models.load_model('/content/drive/MyDrive/Amazon ML Challenge/Vishal_MK_model.h5')

In [21]:
test_data = pd.read_csv('/content/drive/MyDrive/Amazon ML Challenge/dataset/test.csv', escapechar = "\\", quoting = csv.QUOTE_NONE)

In [28]:
features = test_data['TITLE'].replace(np.nan, '', regex = True) + ' ' + test_data['DESCRIPTION'].replace(np.nan, '', regex = True) + ' ' + test_data['BULLET_POINTS'].replace(np.nan, '', regex = True)

In [31]:
encoded_docs = tokenizer.texts_to_sequences(features)

In [32]:
padded_docs = sequence.pad_sequences(encoded_docs, padding = 'post')

In [33]:
X_test_encoded = np.array(padded_docs)

In [34]:
X_test_encoded.shape

(110775, 828)

In [38]:
predictions = np.argmax(model.predict(X_test_encoded), axis=-1)

In [40]:
submissions = pd.DataFrame({'PRODUCT_ID' : test_data['PRODUCT_ID'], 'BROWSE_NODE_ID' : predictions})

In [43]:
submissions['BROWSE_NODE_ID'] = submissions['BROWSE_NODE_ID'].map(lambda x: map_cat_to_targets[x])

In [45]:
submissions.to_csv('Submissions.csv', index = False)